In [1]:
import torch
from importlib import reload
import torchaudio
reload(torchaudio)


train_dataset = torchaudio.datasets.LIBRISPEECH('.',
url='train-clean-100', download=False)                      # pairs of waveforms and text transcription
val_dataset = torchaudio.datasets.LIBRISPEECH('.',
url='dev-clean', download=False)
test_dataset = torchaudio.datasets.LIBRISPEECH('.',
url='test-clean', download=False)

In [2]:
train_dataset.__getitem__(1)

(tensor([[-0.0059, -0.0045, -0.0067,  ...,  0.0007,  0.0034,  0.0047]]),
 16000,
 "THAT HAD ITS SOURCE AWAY BACK IN THE WOODS OF THE OLD CUTHBERT PLACE IT WAS REPUTED TO BE AN INTRICATE HEADLONG BROOK IN ITS EARLIER COURSE THROUGH THOSE WOODS WITH DARK SECRETS OF POOL AND CASCADE BUT BY THE TIME IT REACHED LYNDE'S HOLLOW IT WAS A QUIET WELL CONDUCTED LITTLE STREAM",
 103,
 1240,
 1)

In [84]:
chars = '’ abcdefghijklmnopqrstuvwxyz'
print(len(chars), chars[27])
code_char_dict = {k: c for k, c in enumerate(list(chars))}
code_char_dict

28 z


{0: '’',
 1: ' ',
 2: 'a',
 3: 'b',
 4: 'c',
 5: 'd',
 6: 'e',
 7: 'f',
 8: 'g',
 9: 'h',
 10: 'i',
 11: 'j',
 12: 'k',
 13: 'l',
 14: 'm',
 15: 'n',
 16: 'o',
 17: 'p',
 18: 'q',
 19: 'r',
 20: 's',
 21: 't',
 22: 'u',
 23: 'v',
 24: 'w',
 25: 'x',
 26: 'y',
 27: 'z'}

#### Audio transforms for data preparation

In [40]:
from importlib import reload
import lab4_proto
reload(lab4_proto)
from lab4_proto import *


train_audio_transform = ['augment']
test_audio_transform = []
dev_audio_transform = []

In [23]:
example = torch.load('lab4_example.pt')
for el in example:
    # data, spectrograms, labels, input_lengths, label_lengths
    print(el)

data
spectrograms
labels
input_lengths
label_lengths


In [28]:
print(len(example['data']))
print(example['spectrograms'].shape)
print(example['labels'].shape)
print(len(example['input_lengths']), len(example['label_lengths']))

5
torch.Size([5, 1, 80, 835])
torch.Size([5, 158])
5 5


In [105]:
import torch.nn.functional as F
import torch.utils.data as data


# test the data loader
use_mps = False
kwargs = {'num_workers': 1, 'pin_memory': True} if use_mps else {}
train_loader = data.DataLoader(dataset=train_dataset,
                               batch_size=256,
                               shuffle=True,
                               collate_fn=lambda x: dataProcessing(x, train_audio_transform),
                               **kwargs)


for batch in train_loader:
    print(batch)
    break

print(len(batch))

(tensor([[[[2.5657e-07, 2.4250e-06, 7.6435e-06,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [9.2881e-07, 8.7785e-06, 2.7670e-05,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [5.4415e-05, 1.2412e-04, 4.2625e-04,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [4.4494e-05, 1.2057e-05, 1.8375e-05,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [1.8472e-05, 1.4888e-05, 3.4767e-05,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [1.3438e-05, 1.9748e-05, 9.1366e-06,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00]]],


        [[[2.6588e-03, 2.6696e-03, 2.0620e-03,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [9.6249e-03, 9.6643e-03, 7.4645e-03,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          [2.7526e-06, 1.2681e-03, 7.9891e-04,  ..., 0.0000e+00,
           0.0000e+00, 0.0000e+00],
          ...,
          [2.7875e-05, 2.1275e-05, 2.7799e-05,  ..., 0.0

In [56]:
print(batch[0].shape)
print(batch[1].shape)
print(len(batch[2]), len(batch[3]))

torch.Size([256, 1, 80, 1349])
torch.Size([256, 301])
256 256


In [13]:
!pip install kenlm

  Using cached kenlm-0.2.0.tar.gz (427 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for kenlm: filename=kenlm-0.2.0-cp310-cp310-macosx_14_0_arm64.whl size=484353 sha256=5d1e16f3c460eebe14bfa65d563bedc6d92dc578f993ea2be31ac8fa875e824c
  Stored in directory: /Users/annkle/Library/Caches/pip/wheels/fd/80/e0/18f4148e863fb137bd87e21ee2bf423b81b3ed6989dab95135
Successfully built kenlm


In [21]:
!python lab4_main.py --mode test --model 'checkpoints/epoch-12.pt'

Using device: cpu
SpeechRecognitionModel(
  (cnn): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (rescnn_layers): Sequential(
    (0): ResidualCNN(
      (cnn1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (layer_norm1): CNNLayerNorm(
        (layer_norm): LayerNorm((40,), eps=1e-05, elementwise_affine=True)
      )
      (layer_norm2): CNNLayerNorm(
        (layer_norm): LayerNorm((40,), eps=1e-05, elementwise_affine=True)
      )
    )
    (1): ResidualCNN(
      (cnn1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (layer_norm1): CNNLayerNorm(
        (layer_norm)

Results on test with **default** alpha and beta values:

```
Average CTC loss: 0.6938

Test set with Greedy Decoder: Average CER: 0.152721 Average WER: 0.5541

Test set with pyCTC Decoder: Average CER: 0.169338 Average WER: 0.3616
```

In [66]:
import lab4_proto
reload(lab4_proto)
from lab4_proto import *


levenshteinDistance('HF J', 'JJHJ')

3.0

In [10]:
!pip install pyctcdecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.2/465.2 kB 3.9 MB/s eta 0:00:0000:0100:01


In [20]:
from importlib import reload
import lab4_main
reload(lab4_main)
from lab4_main import cer, wer

targets_batch = ['D E A R E S T   T E A C H   M E   S O   T O   P O U R   O U T   G R A T I T U D E   A S   T H O U   D O S T   G O O D', 'M U S S U L M A N S   A N D   G I A O U R S   T H R O W   K E R C H I E F S   A T   A   S M I L E   A N D   H A V E   N O   R U T H   F O R   A N Y   W E E P I N G', 'B U T   T H O U   A R T   N O T   S U C H   A   L O V E R   M Y   B E L O V E D', 'T H O U   C A N S T   W A I T   T H R O U G H   S O R R O W   A N D   S I C K N E S S   T O   B R I N G   S O U L S   T O   T O U C H   A N D   T H I N K   I T   S O O N   W H E N   O T H E R S   C R Y   T O O   L A T E', 'I   T H A N K   A L L   W H O   H A V E   L O V E D   M E   I N   T H E I R   H E A R T S   W I T H   T H A N K S   A N D   L O V E   F R O M   M I N E', "O H   T O   S H O O T   M Y   S O U L ' S   F U L L   M E A N I N G   I N T O   F U T U R E   Y E A R S   T H A T   T H E Y   S H O U L D   L E N D   I T   U T T E R A N C E   A N D   S A L U T E   L O V E   T H A T   E N D U R E S   F R O M   L I F E   T H A T   D I S A P P E A R S", "T H E N   I   L O N G   T R I E D   B Y   N A T U R A L   I L L S   R E C E I V E D   T H E   C O M F O R T   F A S T   W H I L E   B U D D I N G   A T   T H Y   S I G H T   M Y   P I L G R I M ' S   S T A F F   G A V E   O U T   G R E E N   L E A V E S   W I T H   M O R N I N G   D E W S   I M P E A R L E D", 'I   L O V E   T H E E   F R E E L Y   A S   M E N   S T R I V E   F O R   R I G H T   I   L O V E   T H E E   P U R E L Y   A S   T H E Y   T U R N   F R O M   P R A I S E', "I   L O V E   T H E E   W I T H   T H E   P A S S I O N   P U T   T O   U S E   I N   M Y   O L D   G R I E F S   A N D   W I T H   M Y   C H I L D H O O D ' S   F A I T H", 'I   L O V E   T H E E   W I T H   A   L O V E   I   S E E M E D   T O   L O S E   W I T H   M Y   L O S T   S A I N T S   I   L O V E   T H E E   W I T H   T H E   B R E A T H   S M I L E S   T E A R S   O F   A L L   M Y   L I F E   A N D   I F   G O D   C H O O S E   I   S H A L L   B U T   L O V E   T H E E   B E T T E R   A F T E R   D E A T H']
pyctc_decoded_batch = ['dear ist teach me so to poor outcratitud as thou dust got', 'musslmens and diars drill courses at a small and half no left for any leaping', 'but it owl are not such a lever my the loved', 'thowcancet wait through sorrow and sickness to bring souls to touch and think it cue when others cry too late', 'i think all who have love me in their hearts with fans and love from mine', 'oh to sod my sells full meaning into future hears that they should end in utterances and silent love that indoors from life that disappears', 'than i entriaed by naturally olds receive the comfort fast will body at the sight my pilgrims staff gave out green ties with morning des impurle', 'i love be freely as men strive for rite i love be purely as they turned from prays', 'i love be with the passion put to use in my old greats in with my child has fat', 'i leave the with a love i seemed to loan with my lost cans i love with the breath smile tear of all my life and if do this i show but loved the better after death']
greedy_decoded_batch = ['D E A R   I S E D   T E A C H   M E   S O   T O   P O R   O U T   C R A T I T U D   A S   T H O W N   D U S T   G O O I T', 'M U S S L M E N S   A N D   D I A R S   T H R I L   C I R C H I V S   A T   A S   S M A L L L   A N D   H A L F   N O   L I E A T H   F O R   A N Y   L E A P I N G', 'B U T   I T   H O U L   A R   N O T   S U C H   A   L E T V E R   M Y   T H E   L O I E D', "T H O W   C A N C E T   W A I T E   T H R O U G H   S O R O W   A N D   S I C K N E S S   T O   B R I N G   S O U L S   T O   T O U C H   A N D   T H I N K   I T X U I V E   W H E N   O T H E R ' S   C R Y   T O W O   L A", 'I   T H I N K   A L   W H O   H A V E   L E V E D   M E   I N   T H E I R   H E A R T S   W I H   F A N K S   A N D   L O V E   F R O M   M I N E', 'O H   T O   S H O D   M Y   S E U L L E S   F U L L   M E A N I N G   I N T O   F U A T U R E   E R S   T H A T   T H E Y   S H O U L D   W E N D   I T   U T D E R E C S   A N D   S O L E A T   L H O F E   T H A T   I N D E O R S   F R O M   L I F E   T H A T   D I S A P E E R S', 'T H A N   I   R E N E   T R I A E D E   B Y   N A T U R A L L Y A L S   R E   C E I E   T H E   C O M F O R T   F A S T   W H I L L   B U T D Y   A T   T H Y   S T I G D T   M Y   P O L L G L I N M D   S T A F   G A V E   O U T   G R E E M   L T H E S   W I T H   M O R N I N G   D E D S   I M P U R L E', 'I   L E V E   B E   F R E E L Y   I S   M E N   S T R Y E   F O R   R I H E   I   L O E V E   B E   E F U R E D L Y   A S   T H E Y   T U R N E D   F R O M   P R A S', 'I   L O V E D   B E   W I T H   T H E   P A S H I O N   P U T T O   U T H S E S   I N   M Y   O L D   G R E E S   I N   W I T H   M Y   C H I L E D   H O O D S   F A', 'I   L E E   B H E   W I T H   A   L O V E   I S E E M E D   T O   L E A   W I T H   M Y   L O S T   S A N S   I   L O V E   B E   W I T H   T H E   B R E A T H   S M I L E   T E A R S   O F   A L L   M I T   L I F E   A N D   I F   D O D   T H E S   I   S H O W   B U T   L O V E   T E   B E T T E R   A F T E R   D E A T G H']

wers_greedy, wers_pyctc = [], []
for item in zip(targets_batch, greedy_decoded_batch, pyctc_decoded_batch):
    wers_greedy.append(wer(item[0], item[1], ignore_case=True, greedy=True))
    wers_pyctc.append(wer(item[0], item[2], ignore_case=True))

print(wers_greedy)
print(wers_pyctc)

[0.5833333333333334, 0.6, 0.6666666666666666, 0.42857142857142855, 0.3125, 0.4, 0.5555555555555556, 0.5555555555555556, 0.6111111111111112, 0.3684210526315789]
[0.5833333333333334, 0.5333333333333333, 0.6666666666666666, 0.14285714285714285, 0.1875, 0.32, 0.48148148148148145, 0.2777777777777778, 0.3333333333333333, 0.3157894736842105]


### Comparing results with and without language model:

Subjectively, adding language models obviously recognizes better some words, like turnine "ave" into "of", "P R O S E S S" into "proves" (however, the target was "proves this") etc. So it's good for shaping words that almost right into grammatical form wihtout misspellings, but sometimes it predicts it wrong. For example, it's bad for recognizing unique names.


After training for 13 epochs:

```
Average loss: 0.6938

Test set with Greedy Decoder: Average CER: 0.152721 Average WER: 0.5541

Test set with pyCTC Decoder: Average CER: 0.169338 Average WER: 0.3616
```

WER is much lower for pyCTCDecoder. CER is comarable for both decoders, but a little better for GreedyDecoder.


In [23]:
import numpy as np
np.linspace(0.05, 1, 5)

array([0.05  , 0.2875, 0.525 , 0.7625, 1.    ])

In [55]:
!python lab4_main.py --mode tunedecoder

Using device: cpu
SpeechRecognitionModel(
  (cnn): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (rescnn_layers): Sequential(
    (0): ResidualCNN(
      (cnn1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (layer_norm1): CNNLayerNorm(
        (layer_norm): LayerNorm((40,), eps=1e-05, elementwise_affine=True)
      )
      (layer_norm2): CNNLayerNorm(
        (layer_norm): LayerNorm((40,), eps=1e-05, elementwise_affine=True)
      )
    )
    (1): ResidualCNN(
      (cnn1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (layer_norm1): CNNLayerNorm(
        (layer_norm)

Best alpha and beta (run on cuda):

```
Best alpha: 0.525, Best beta: 1.0, Best WER: 0.3805722415694304
```

Evaluating the model with the tuned parameters on the test set:

In [30]:
!python lab4_main.py --mode testlm --model 'checkpoints/epoch-12.pt'

Using device: cpu
len(val_dataset), len(test_dataset):  2703 2620
SpeechRecognitionModel(
  (cnn): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (rescnn_layers): Sequential(
    (0): ResidualCNN(
      (cnn1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (layer_norm1): CNNLayerNorm(
        (layer_norm): LayerNorm((40,), eps=1e-05, elementwise_affine=True)
      )
      (layer_norm2): CNNLayerNorm(
        (layer_norm): LayerNorm((40,), eps=1e-05, elementwise_affine=True)
      )
    )
    (1): ResidualCNN(
      (cnn1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (

Scores without language model:

```
Average test loss: 0.6938

Test set with Greedy Decoder: Average CER: 0.152721 Average WER: 0.5541
```

Non optimized decoder = Average CER: 0.169338 Average WER: 0.3616

Scores with language model on tuned alpha beta for pyCTC decoder:

```
Test set with optimized pyCTC Decoder: 

Average test loss: 0.6938

Average CER: 0.169864 Average WER: 0.3609
```

Opitimized pyCTC decoder is only a tiny bit better on WER than a non-optimized one, because after grid search the optimized alpha and beta turned out to be very close to default ones.


Try your model on a few different speech files: download from internet, or record your own! How does it perform?

In [66]:
!python lab4_main.py --mode recognize --model 'checkpoints/epoch-12.pt' --wavfiles 19.wav 20.wav

Using device: cpu
SpeechRecognitionModel(
  (cnn): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (rescnn_layers): Sequential(
    (0): ResidualCNN(
      (cnn1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (layer_norm1): CNNLayerNorm(
        (layer_norm): LayerNorm((40,), eps=1e-05, elementwise_affine=True)
      )
      (layer_norm2): CNNLayerNorm(
        (layer_norm): LayerNorm((40,), eps=1e-05, elementwise_affine=True)
      )
    )
    (1): ResidualCNN(
      (cnn1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (layer_norm1): CNNLayerNorm(
        (layer_norm)

**wavfile: 19.wav**

```
text: ['  O I Y   I   O W M Y S   T L O O N     W T   S O   M O N   G O O N   O   O N I N   O E   T R O E N G W H E     M O R D   R U   O   P R O O       O    ']

pyctcdecoded_preds:  ['ii ones to got so on grown or one or throne marred roof poor to h']

target: i also understand that similar branch organizattions have made their appearance in europe
```

**wavfile: 20.wav**

```
text: ['  E T   G O O   M A   H A N T O   C O O M   M E R O N   C O R B E   W H O D O D   N E   C O   O D   G O   A G E H D   T   C R U L   T R E O   H O N D   H O L R Y W S O N   O   N     F O  ']

pyctcdecoded_preds: ['et gone mare anta come care who did not core or grunge tag proud to und holeson on from i']

target: a combination of canadian capital quickly organized and petioned for the same privileges
```

It performs much worse on recordings of my voice. I think it's because the quality of recording and conditions are completely different.